In [77]:
from numpy import exp, array, random, dot
import numpy as np
from scipy import misc
import skimage
import math
from __future__ import with_statement

In [5]:
arr1 = np.divide(misc.imread('tAmerica1.png'),255)
arr1r = arr1[:,:,0]
arr1g = arr1[:,:,1]
arr1b = arr1[:,:,2]
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [6]:
filterNum = 3
filterSize = 10
layerNum = 6

In [7]:
filters = []
for x in range(filterSize*layerNum):
    filters += [0.01*random.random((filterSize,filterSize))-0.005] #run previous before this 
filters = array(filters)
print(filters)

[[[ -2.90704921e-03   4.31939360e-03   4.76084879e-03 ...,   4.05232719e-03
    -4.25551373e-03  -1.74561574e-04]
  [ -3.33877147e-03   2.48664564e-03   3.25049211e-03 ...,  -6.87861276e-04
     3.83933150e-03   2.99806384e-03]
  [  2.84952725e-03  -4.97684607e-03   4.08086413e-03 ...,  -1.09124265e-03
     7.57698387e-04   3.13135966e-03]
  ..., 
  [ -2.25331682e-03  -4.14080269e-03  -3.26554466e-03 ...,   3.77753063e-04
    -4.58795603e-03   3.49673187e-03]
  [  4.81065796e-03   8.56679421e-04   2.11700218e-03 ...,  -4.97525353e-03
     3.80250080e-03  -5.18145859e-04]
  [  4.51916273e-03  -3.23195943e-03  -6.95548152e-04 ...,   4.96712243e-03
    -4.92587590e-03  -3.27080402e-03]]

 [[ -3.54628045e-04   2.21736043e-03  -1.08943604e-03 ...,   3.02928139e-03
    -2.68023704e-03  -3.41246072e-03]
  [  5.95233065e-04  -2.95542077e-03   3.08777959e-03 ...,   3.44802689e-03
     4.89892135e-03  -2.24163865e-03]
  [ -1.49233123e-03   3.56110052e-03  -4.71936161e-03 ...,   1.45917687e-04
  

In [99]:
def activation(x, deriv=False):
    if deriv:
        return (x > 0)
    return x * (x > 0)
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
def apply(filterArr, img, row, col):
    value = 0
    halfFil = int(filterSize/2)
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0))
    for y in range(len(filterArr)):
        for x in range(len(filterArr[0])):
            value += padImg[y+row-halfFil][x+col-halfFil]*filterArr[y][x]
    return value/sumNoNeg(filterArr)
print(apply(filters[4],arr1r,0,0))

0.0


In [100]:
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])):
            new_img[y][x] += activation(apply(filterArr,img,y,x))
    return new_img
# print(len(([[0]*(len(arr1r))]*(len(arr1r[0])))[0]))
# print(len(arr1r[0]))
# #print([[0]*(len(arr1r)-filterSize)]*(len(arr1r[0])-filterSize))
# print(applyFilters(arr1r, np.zeros((len(arr1r)-filterSize, len(arr1r[0])-filterSize)), filters[0]))

In [101]:
def forward(img):
    outputLayers = [img]
    for num in range(layerNum):
        newImage = np.zeros((len(outputLayers[num]), len(outputLayers[num][0])))
        for fil in range(filterNum):
            applyFilters(outputLayers[num], newImage, filters[filterNum*num + fil])
            print(filterNum*num + fil)
            print(newImage)
        outputLayers.append(np.divide(newImage, filterNum))
    return outputLayers
out = forward(arr1r)
print(out)

0
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.00641821  0.0111517  ...,  0.          0.          0.        ]
 [ 0.          0.01496279  0.02369498 ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.06092263  0.03373236 ...,  0.          0.          0.        ]
 [ 0.          0.05359083  0.03143737 ...,  0.          0.          0.        ]
 [ 0.          0.05359083  0.03143737 ...,  0.          0.          0.        ]]
1
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.00641821  0.01281675 ...,  0.          0.          0.        ]
 [ 0.          0.01496279  0.02369498 ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.06092263  0.03373236 ...,  0.04995685  0.04995685
   0.04995685]
 [ 0.          0.05359083  0.03143737 ...,  0.06447092  0.06447092
   0.06447092]
 [ 0.          0.05359083  0.03143737 ...,  0.06447092  0.06447092
   0.06447092]]
2
[[ 0.       

In [88]:
def maxPool(size, img):
    new_img = np.zeros((int(len(img)/size), int(len(img[0])/size)))
    pad_img = np.pad(img, [(0, size-len(img)%size),(0, size-len(img[0])%size)], "constant", constant_values=(0))
    for x in range(math.ceil(len(img)/size)-1):
        for y in range(math.ceil(len(img[0])/size)-1):
            new_img[x][y] = np.amax(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
            print(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
    return new_img
print(maxPool(3, out[2]))

[[ 0.00961632  0.0087006   0.00945776  0.00782783  0.00715145  0.00502169
   0.00435108  0.00413986  0.00291629  0.00232335  0.00281515  0.00296963
   0.00295165  0.00337384  0.00450634  0.00475051  0.00692215  0.010059
   0.01323215  0.01362665  0.00971161  0.0086846   0.00929713  0.00774717
   0.00711855  0.00502169  0.00435108  0.00413986  0.00291629  0.00232335
   0.00281515  0.00296963  0.00295165  0.00337384  0.00450634  0.00475051
   0.00692215  0.010059    0.01323215  0.01362665  0.00971161  0.0086846
   0.00929713  0.00774717  0.00711855  0.00502169  0.00435108  0.00413986
   0.00291629  0.00232335  0.00281515  0.00296963  0.00295165  0.00337384
   0.00450634  0.00475051  0.00692215  0.010059    0.01323215  0.01362665
   0.00971161  0.0086846   0.00929713  0.00774717  0.00711855  0.00502169
   0.00435108  0.00413986  0.00291629  0.00232335  0.00281515  0.00296963
   0.00295165  0.00337384  0.00450634  0.00475051  0.00692215  0.010059
   0.01323215  0.01362665  0.00971161  0.00

ValueError: zero-size array to reduction operation maximum which has no identity

In [92]:
def error(output):
    return 1-np.mean(output)
def back(img, startlayer, error):
    print(startlayer)
    newout = np.zeros((len(out[startlayer-1]), len(out[startlayer-1][0])))
    if startlayer == -1:
        return out[0]
    applyFilters(back(img, startlayer-1, error),
                 newout,
                 np.multiply(np.rot90(np.rot90(filters[startlayer])),activation(img,deriv=True)[:filterSize,:filterSize]))
    return newout
backprop = back(out[2], 2, error(out[layerNum]))

2
1
0
-1


IndexError: index 240 is out of bounds for axis 0 with size 240